In [7]:
import pandas as pd
import numpy as np
import json
import ast
from tqdm import tqdm

# from src.Basic_LLM_functions import LLM_connection
from src.Remove_accents import remove_accents,remove_special_characters,divide_string_into_words
from src.Spanish_words import spanish_words
from src.Basic_LLM_functions import LLM_connection

In [2]:
def dict_to_string(d):
    return "\n".join([f"{k}: {v}" for k,v in d.items()])

In [3]:
df=pd.read_csv("Databases/Parsed_htmls.csv")

In [4]:
#Obtenido tras simplificar new_keys yo mismo
final_keys={
    'accesorios'
    'Tiene ahorro energia',
    'alcance',
    'alimentacion',
    'almacenamiento',
    'amperaje',
    'apagado automatico',
    'apariencia',
    'aplicaciones',
    'autonomia',
    'bateria',
    'Tiempo de carga',
    'bolsa',
    'cabezal',
    'capacidad del deposito de polvo',
    'cepillo',
    'clase de reemision de polvo',
    'color',
    'componentes',
    'conectividad',
    'consumo',
    'detalles fisicos adicionales',
    'dimensiones del producto',
    'peso',
    'filtro',
    'funcion seca suelo',
    'garantia',
    'iluminacion',
    'inalambrico',
    'longitud cable',
    'marca',
    'material',
    'maxima presion de succion',
    'nivel de ruido',
    'modos de funcionamiento',
    'motor',
    'necesita baterias',
    'se incluyen las pilas',
    'tipos de superficie',
    'caracteristicas adicionales'
    }

In [5]:
LLM_connection.load_model_from_path("src/LLMs/Meta-Llama-3.1-8B-Instruct")

c:\Users\Rafa\miniconda3\envs\llama3_1_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.49s/it]


In [9]:
LLM_connection.system_prompt=f"""You will be given information about a product and you will try to extract the following information into the following json format:
                                {"{"+":, \n".join(list(final_keys))+"}"}

                                If no information on a key is provided, just leave it an empty string ("")
                                Answer only with the dictionary.
                                """

In [46]:
import ast
def turn_info_to_json(text,max_new_tokens=10000):
    for i in range(3):
        out=LLM_connection.Get_answer(text)
        try:
            output=ast.literal_eval(out[out.index("{"):].replace("`","").replace("`",""))
            return output
        except:
            pass
    return {}

In [10]:
def turn_info_to_json_batch(questions, max_new_tokens=10000):
    results = [None] * len(questions)  # Initialize results with None for tracking
    remaining_indices = list(range(len(questions)))  # Track indices of questions to process
    
    for attempt in range(3):  # Retry mechanism, up to 3 attempts
        if not remaining_indices:  # Stop if all questions are successfully processed
            break

        # Fetch answers for remaining questions
        current_questions = [questions[i] for i in remaining_indices]
        responses = LLM_connection.Get_answer_batch(current_questions)
        
        # Process each response
        new_remaining_indices = []
        for idx, response in zip(remaining_indices, responses):
            try:
                # Attempt to parse the response
                output = ast.literal_eval(response[response.index("{"):].replace("`", "").replace("`", ""))
                results[idx] = output  # Store successful result
            except:
                new_remaining_indices.append(idx)  # Retain index for retry
        
        # Update remaining indices
        remaining_indices = new_remaining_indices

    # Fill remaining failed indices with empty dictionaries
    for idx in remaining_indices:
        results[idx] = {}

    return results

In [11]:
questions=["General Information:\n"+f"""{df.at[i,"General Information"]}"""+"\n"+\
                            "About This Product:\n"+"\n".join(ast.literal_eval(df.at[i,"About This Product"]))+"\n"+\
                            "Technical Details:\n"+dict_to_string(ast.literal_eval(df.at[i,"Technical Details"]))
                            for i in df.index]

In [13]:
final_jsons=turn_info_to_json_batch(questions)

In [39]:
def unify_columns(df):
    """
    Unify columns in a DataFrame by:
    - Removing accents from column names.
    - Converting column names to lowercase.
    - Merging duplicate columns by taking the first non-null value in each row.
    """
    df.columns = [remove_accents(col).lower() for col in df.columns]
    df_unified = pd.DataFrame(index=df.index)
    
    unique_columns = df.columns.unique()
    
    for col in unique_columns:
        cols = df.loc[:, df.columns == col]

        if cols.shape[1] == 1:
            df_unified[col] = cols.iloc[:, 0]
        else:
            # Merge columns by taking the first non-null value in each row
            df_unified[col] = cols.apply(lambda x: next((item for item in x if pd.notna(item)), np.nan), axis=1)
    
    return df_unified

In [40]:
jsons_df=unify_columns(pd.DataFrame(final_jsons))

In [44]:
def extract_and_divide(star_string):
    if "de 5 estrellas" in star_string:
        try:
            # Replace ',' with '.' for decimal conversion and extract the first number
            return float(star_string.split(" de ")[0].replace(",", ".")) / 5
        except ValueError:
            return None
    else:
        return None

In [63]:
df["Stars"]=df["Stars"].apply(extract_and_divide)

In [59]:
def format_price(value):
    if "Price not found" in value:
        return "-1"
    else:
        # Remove the currency symbol and replace comma with a dot
        numeric_part = value.replace("€", "").replace(",", ".")
        return f"{numeric_part}"

In [61]:
df["Price"]=df["Price"].apply(format_price).astype(float)

In [66]:
full_df=pd.concat([df.drop(columns=['General Information', 'About This Product','Technical Details']),jsons_df],axis=1)

In [68]:
full_df["All_info"]=questions

In [78]:
full_df.to_csv("Databases/Cleaned_df.csv",index=False)